In [1]:
import os
import sys
import glob
import datetime
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xray
import random
import itertools
import urllib
import json
import importlib as imp

from IPython import display

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.dates as climadates
import climatools.viz as climaviz
import climatools.misc as climamisc
import climatools.muths as climaths
import climatools.units as climaunits
import climatools.geocoords as climageocoords
import climatools.prepdatasets as climaprepdata

sys.path.append('/nuwa_cluster/home/jackyu/radiation/rtmtools/')
import rtmtools.rrtmg.sw.info as rrtmgswinfo

#from IPython.lib.deepreload import reload as dreload
imp.reload(climadates)
imp.reload(climaviz)
imp.reload(climamisc)
imp.reload(climaths)
imp.reload(climaunits)
imp.reload(climaprepdata)

imp.reload(rrtmgswinfo)


matplotlib.rcParams.update({'font.size': 13, 
                            'xtick.direction': 'out',
                            'ytick.direction': 'out'})

# ARM95

2 cases considered here are:

* a McICA --- RRTMG with McICA cloud overlapping. Output sent to CAM
* b MaxRandCldscaleX --- RRTMG with MaxRand cloud overlapping, where, in the longwave, cloud optical depth is scaled by conserving transmittance.  Output sent to CAM.
 

In [3]:

iopname = 'arm95_McICA_vs_MaxRandCldscaleX'

def case_names():
    'Give the different cases names'
    return ['a McICA', 'b MaxRandCldscaleX']



def get_cases_difference():
    '''
    define the differences between cases to compute
    '''
    return [('b MaxRandCldscaleX', 'a McICA'),]



def get_cases():
    '''
    Returns a dictionary where the keys are cases, such
    as a name for a SCAM run, and the values are the datasets (xray)
    for the cases
    '''
    ds_a = xray.open_dataset('../examples/ARM95/cam5_noavg_LW_rrtmgMcICA_rrtmgMaxRand_SW_rrtmgMcICA_rrtmgMaxRand/camrun.cam.h0.0095-07-18-19800.nc', 
                                decode_times = False)
    ds_b = xray.open_dataset('../examples/ARM95/cam5_noavg_LW_rrtmgMaxRandCldScaleX_SW_rrtmgMaxRand/camrun.cam.h0.0095-07-18-19800.nc',
                                  decode_times = False)    
    return dict(zip(case_names(), [ds_a, ds_b]))




def compute_net_solar_fluxes(datasets):
    '''
    Compute net solar fluxes
    '''
    for casename, ds in datasets.items():
        ds['net_FS'] = ds['FDS'] - ds['FUS']
        ds['net_FS'].attrs['units'] = ds['FDS'].attrs['units']
        ds['net_FS'].attrs['long_name'] = 'net solar flux'
        ds['net_FS'].attrs['case_name'] = casename
        
        ds['net_FSC'] = ds['FDSC'] - ds['FUSC']
        ds['net_FSC'].attrs['units'] = ds['FDSC'].attrs['units']
        ds['net_FSC'].attrs['long_name'] = 'net solar flux (clear-sky)'
        ds['net_FSC'].attrs['case_name'] = casename
    return datasets




def linestyles_by_case():
    '''
    Assign lineplot properties to the cases
    '''
    return dict(zip(case_names(), 
                    [{'colour': 'g', 'linestyle': '--'},
                     {'colour': 'b', 'linestyle': '--'}]))



def get_d3interests():
    '''
    List here all variables of interest that have dimensions (time, lev, lon, lat)
    '''
    d3interests = ['CLOUD', 'QRL', 'QRS', 'Q',
                   'CLDICE', 'CLDLIQ', 
                   'AQSNOW', 'AREI', 'AREL',
                   'AWNC', 'AWNI', 
                   'DCQ', 'DTCOND', 
                   'ICIMR', 'ICWMR', 
                   'NUMLIQ', 'NUMICE', 'T', 'RELHUM',
                   'FUL', 'FDL', 'FULC', 'FDLC', 'net_FS', 'net_FSC',
                   'FUS', 'FDS', 'FUSC', 'FDSC',
                   'TOT_CLD_VISTAU']
    return sorted(d3interests)




def get_d2interests():
    '''
    List here all variables of interest that have dimensions (time, lon, lat)
    '''
    interests = ['FSNT', 'FSNS', 'FLNT', 'FLNS',
                 'FSNTC', 'FSNSC', 'FLNTC', 'FLNSC',
                 'CLDLOW', 'CLDMED', 'CLDHGH', 'CLDTOT',
                 'TS', 'TREFHT', 'TROP_T', 'TROP_Z', 'TMQ', 'TGCLDIWP', 'TGCLDLWP', 
                 'PRECT', 'PRECL', 'PRECC',
                 'LHFLX', 'SHFLX',
                'SOLIN']
    return interests






def convert_units(datasets):
    '''
    Convert units for certain fields for all cases in DATASETS.
    INPUT:
    ilev --- level(interface) pressure in mbar
    lev  --- layer(level) pressure in mbar
    datetimes --- time in pandas DatetimeIndex objects
    '''
    for name, ds in datasets.items():
        
        for k in ['TOT_CLD_VISTAU',]:
            if k in ds:
                pass #ds[k].values[ds[k].isnull().values] = - 0.
        
        for k in ['FUS', 'FDS', 'net_FS', 'net_FSC']:
            if k in ds:
                ds[k] *= 1e3
    
        for k in ['QRL', 'QRS', 'DTCOND']:
            if k in ds:
                ds[k] *= 86400
                ds[k].attrs['units'] = 'K/day'
        
        for k in ['Q', ]:
            if k in ds:
                ds[k] *= 1e3
                ds[k].attrs['units'] = 'g/kg'
        
        for k in ['CLDICE', 'CLDLIQ', 'AQSNOW']:
            if k in ds:
                ds[k] *= 1e6
                ds[k].attrs['units'] = 'mg/kg'
                
        for k in ['PRECC', 'PRECL', 'PRECT']:
            if k in ds:
                ds[k] *= 1e3 * 86400
                ds[k].attrs['units'] = 'mm/day'
                
    return datasets

In [4]:
def prep_data(cases):
    '''
    Preprocess data for plotting. 
    pressure and datetimes need to be provided for unit conversion.
    INPUT:
    ilev --- level(interface) pressure [mbar]
    lev --- layer(level) pressure [mbar]
    datetimes --- time in pandas DatetimeIndex objects    
    '''
    interests = get_d3interests() + get_d2interests()
    cases = climaprepdata.convert_pressure_time_units(cases)
    cases = climaprepdata.gather_interests_from_cases(cases, interests)
    cases = convert_units(cases)
    diff_cases = climaprepdata.take_difference_between_cases(cases, get_cases_difference())
    cases, diff_cases = climaprepdata.passon_attrs_casename(cases, diff_cases, interests = interests)
    return cases, diff_cases


In [5]:
################## Variables of dimensions (time, lev, lon, lat) ########################
#########################################################################################


# for writing plotted figures to disk
def writefig_contourfs(d3sets, diff_d3sets, interests, 
                       dir_label = 'example_dir_label', file_label = 'example_file_label',
                       fmt = 'png'):
    '''
    For each interest, produce one figure containing contour
    plots for all cases, another figure containing contour 
    Hackish colormap selection here, basically look up a dictionary.
    '''

    for interest in interests:
        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)

            
        filename = '_'.join([interest, file_label]) if file_label else interest
            
        fig = climaviz.contourf_interest_for_all_cases(d3sets, interest,
                                           cmap = 'auto-select')
        fig.savefig(dirname + '/contourf_{}.{}'.format(filename, fmt))
        plt.close()
        
        fig = climaviz.contourf_interest_for_all_cases(diff_d3sets, interest,
                                           cmap = 'auto-select')
        fig.savefig(dirname + '/contourf_{}_diff.{}'.format(filename, fmt))
        plt.close()
        

        
def writefig_timeaveraged_VSplots(d3sets, diff_d3sets, interests, 
                                 dir_label = 'example_dir_label', file_label = 'example_file_label',
                                 fmt = 'png'):
    '''
    Average over the time dimension, plot against pressure.
    Save figure.
    '''
    for interest in interests:        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
               
        filename = '_'.join([interest, file_label]) if file_label else interest
        
        fig = climaviz.plotVS_timeaveraged_interest_for_all_cases(d3sets, diff_d3sets, 
                               interest = interest, xscale = 'linear',
                               bot_xlabels_rotate = 10., 
                               top_xlabels_rotate = 20.,
                               linestyles = linestyles_by_case())
        fig.savefig(dirname + '/plot_{}_timeaverage.{}'.format(filename, fmt))
        plt.close()
        
        
        
        
def write_d3variable_html_page(interest = 'CLOUD', 
                               dir_label = 'example_dir_label', file_label = 'example_file_label'):
    '''
    Write and save an HTML for interest.  
    Include figures produced for interest: contourfs and time-averaged line plots for example.
    HTML file named by interest and file_label, saved in a directory named by dir_label.
    '''
    figdir = 'figs_' + dir_label
    filename = '_'.join([interest, file_label]) if file_label else interest
    with open(os.path.join(figdir, 'page_{}.html'.format(filename)), 
              mode = 'w', encoding = 'utf-8') as f:
        f.write('''
                <img src="contourf_{suffix}.png" height="450">
                <img src="contourf_{suffix}_diff.png" height="450">
                <br>
                <img src="plot_{suffix}_timeaverage.png" height="500">
        '''.format(figdir = figdir, suffix = filename))

        
        
        
def write_d3variable_links_html(d3sets, interests = ['T'],
                               dir_label = 'example_dir_label', file_label = 'example_file_label'):
    '''
    Write HTML of hyperlinks to HTML files containing figures.
    A hyperlink is written for each interest. 
    '''
    hyperlinks = ['<a href="figs_{dir_label}/page_{filename}.html">{interest}: {longname}</a>'\
                 .format(dir_label = dir_label, 
                         file_label = file_label, 
                         interest = interest, 
                         filename = '_'.join([interest, file_label]) if file_label else interest,
                         longname = d3sets[random.choice(list(d3sets.keys()))][interest].attrs['long_name']) \
                 for interest in interests]
    return '<br>'.join(hyperlinks)
    

In [6]:
############### Variables of dimensions (lon, lat, time) ##########################
###################################################################################


def writefig_plotVS_timeseries(dsets, diff_dsets, interests,
                               dir_label = 'example_dir_label',
                               file_label = 'example_file_label',
                               fmt = 'png'):
    '''
    Plot and save figures of time-series of variables of interest
    '''
    for interest in interests:
        fig = climaviz.plotVS_interest_for_all_cases(dsets, diff_dsets, interest = interest,
                                                    linestyles = linestyles_by_case())
        
        dirname = 'figs_{}'.format(dir_label)
        
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
            
        filename = '_'.join([interest, file_label]) if file_label else interest
            
        fig.savefig(dirname + '/plotVSts_{}.{}'.format(filename, fmt))
        plt.close()


        
        
def write_d2variable_html_page(interest = 'FLNS',
                               dir_label = 'example_dir_label', 
                               file_label = 'example_file_label'):
    '''
    Write and save an HTML for interest.  
    Include figures produced for interest: contourfs and time-averaged line plots for example.
    HTML file named by interest and file_label, saved in a directory named by dir_label.   
    '''
    figdir = 'figs_{}'.format(dir_label)
    filename = '_'.join([interest, file_label]) if file_label else interest
    with open(os.path.join(figdir, 'page_{}.html'\
                           .format(filename)),
             mode = 'w', encoding = 'utf-8') as f:
        f.write('''
        <img src="plotVSts_{}.png" width=700>
        '''.format(filename))
        
        
        

def write_d2variables_html(d2sets, interests = ['FLNS'],
                          dir_label = 'example_dir_label',
                          file_label = 'example_file_label'):
    hyperlinks = ['<a href="figs_{dir_label}/page_{filename}.html">{interest}: {longname}</a>'\
                 .format(dir_label = dir_label, 
                         interest = interest,
                         filename = '_'.join([interest, file_label]) if file_label else interest,
                         longname = d2sets[random.choice(list(d2sets.keys()))]\
                         [interest].attrs['long_name'])\
                 for interest in interests]
    return '<br>'.join(hyperlinks)


In [7]:
def flux_essential_facts(dsets, diff_dsets):
    casea, caseb = case_names()
    case_diff = ' - '.join(get_cases_difference()[0])
    
    d = xray.Dataset()
    
    d['insolation'] = ((), 
                       dsets[casea]['FDS'].isel(lon = 0, lat = 0).\
                       mean(dim = 'time', keep_attrs = True).isel(ilev = 0),
                      dsets[casea]['FDS'].attrs)
        
    d['TOA upward SW ({}) larger by'.format(caseb)] = ((), 
                                           diff_dsets[case_diff]['FUS'][{'lon': 0, 'lat': 0}].\
                                           mean(dim = 'time', keep_attrs = True)[{'ilev': 0}],
                                          diff_dsets[case_diff]['FUS'].attrs)
            
    d['Sfc downward SW ({}) smaller by'.format(caseb)] = ((), 
                                              - diff_dsets[case_diff]['FDS'][dict(lon = 0, lat = 0)].\
                                              mean(dim = 'time', keep_attrs = True)[{'ilev': -1}],
                                             diff_dsets[case_diff]['FDS'].attrs)
        
    d['Atmospheric SW heating ({}) smaller by'.format(caseb)] = ((),
                                                     d['TOA upward SW ({}) larger by'.format(caseb)] \
                                                               - d['Sfc downward SW ({}) smaller by'.format(caseb)],
                                                    d['TOA upward SW ({}) larger by'.format(caseb)].attrs)
    
    
    
    d['OLR ({}) smaller by'.format(caseb)] = ((), 
                                  - diff_dsets[case_diff]['FUL'][dict(lon = 0, lat = 0)].\
                                  mean(dim = 'time', keep_attrs = True).isel(ilev = 1),
                                 diff_dsets[case_diff]['FUL'].attrs)
    
    
    d['TOA (SW+LW) ({}) system cooler by'.format(caseb)] = ((), 
                                                d['TOA upward SW ({}) larger by'.format(caseb)]\
                                                          - d['OLR ({}) smaller by'.format(caseb)],
                                               d['TOA upward SW ({}) larger by'.format(caseb)].attrs)

    
    d['clear sky albedo'] = ((),
                             dsets[caseb]['FUSC'].\
                             isel(lon = 0, lat = 0).mean(dim = 'time', keep_attrs = True).\
                             isel(ilev = 0) / d['insolation'] * 100.,
                            d['insolation'].attrs)
    d['clear sky albedo'].attrs['units'] = '%'
    
    d['insolation (%)'] = ((), d['insolation'] / 1365 * 100, d['insolation'].attrs)
    d['insolation (%)'].attrs['units'] = '%'
    
    
    d['albedo TOA ({}) larger by'.format(caseb)] =  ((),
                                         d['TOA upward SW ({}) larger by'.format(caseb)] / d['insolation'] * 100.,
                                        d['insolation'].attrs)
    d['albedo TOA ({}) larger by'.format(caseb)].attrs['units'] = '%'
    
    
    name_values_units = [(k, da.values, da.attrs['units']) for k, da in d.items()]
    names, values, units = zip(*name_values_units)
    df = pd.DataFrame({'values': np.round(values, 2), 'units': units}, index = names)
    
    ordered_index = ['insolation',
                    'insolation (%)',
                    'clear sky albedo',
                    'TOA upward SW ({}) larger by'.format(caseb),
                    'albedo TOA ({}) larger by'.format(caseb),
                    'Sfc downward SW ({}) smaller by'.format(caseb),
                    'Atmospheric SW heating ({}) smaller by'.format(caseb),
                    'OLR ({}) smaller by'.format(caseb),
                    'TOA (SW+LW) ({}) system cooler by'.format(caseb)]
    return df.loc[ordered_index, ['values', 'units']]



In [8]:
cases = get_cases()
cases = compute_net_solar_fluxes(cases)
dsets, diff_dsets = prep_data(cases) 

# Variables of dimensions (time, ilev, lon, lat)

In [9]:
# plot (time, lev/ilev, lon, lat) variables, write figures to disk 

writefig_contourfs(dsets, diff_dsets, get_d3interests(), 
                   dir_label = iopname, file_label = '',
                  fmt = 'png')

writefig_timeaveraged_VSplots(dsets, diff_dsets, get_d3interests(), 
                              dir_label = iopname, file_label = '',
                              fmt = 'png')

In [10]:
# create HTML files and generate HTML page for each interest
for interest in get_d3interests():
    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = '')


display.HTML(write_d3variable_links_html(dsets, interests = get_d3interests(), dir_label = iopname, file_label = ''))

# Variables of dimensions (lon, lat, time)

In [11]:
# plot (time, lon, lat) variables, and write figures to disk and create HTML files
writefig_plotVS_timeseries(dsets, diff_dsets, get_d2interests(),
                           dir_label = iopname, file_label = '',
                           fmt = 'png')  

In [12]:
# generate HTML page for each interest
for interest in get_d2interests():
    write_d2variable_html_page(interest = interest, dir_label = iopname, file_label = '')



display.HTML(write_d2variables_html(dsets, interests = get_d2interests(),
                                   dir_label = iopname, file_label = ''))

# Flux summary

In [13]:
flux_essential_facts(dsets, diff_dsets)

,values,units
insolation,456.48,W/m2
insolation (%),33.44,%
clear sky albedo,13.75,%
TOA upward SW (b MaxRandCldscaleX) larger by,15.24,W/m2
albedo TOA (b MaxRandCldscaleX) larger by,3.34,%
Sfc downward SW (b MaxRandCldscaleX) smaller by,19.72,W/m2
Atmospheric SW heating (b MaxRandCldscaleX) smaller by,-4.49,W/m2
OLR (b MaxRandCldscaleX) smaller by,1.14,W/m2
TOA (SW+LW) (b MaxRandCldscaleX) system cooler by,14.10,W/m2


In [11]:
def slice_in_time(cases, time = slice(None)):
    return {casename: ds.sel(time = time) for casename, ds in cases.items()}

In [282]:
which_timeslice = slice('1995-07-19 05:00:00', '1995-07-25 05:00:00')

d3sets_subtime = slice_in_time(d3sets, time = which_timeslice)
diff_d3sets_subtime = slice_in_time(diff_d3sets, time = which_timeslice)

In [283]:
# plot and write figures to disk
#interests = get_d3interests()

#writefig_contourfs(d3sets_subtime, diff_d3sets_subtime, interests, 
#                   dir_label = iopname, file_label = which_timeslice,
#                  fmt = 'png')

#writefig_timeaveraged_VSplots(d3sets_subtime, diff_d3sets_subtime, interests, 
#                              dir_label = iopname, file_label = which_timeslice,
#                              fmt = 'png')


In [284]:
# generate HTML page for each interest
#for interest in get_d3interests():
#    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = which_timeslice)


#display.HTML((write_d3variable_links_html(interests = interests, dir_label = iopname, file_label = which_timeslice)))

In [285]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')